## Load libraries

In [2]:
import torch,torchvision
import torchvision.transforms as tr
import matplotlib.pyplot as plt
import numpy as np
import PIL

## Check cuda

In [3]:
!nvidia-smi

Sun Feb 10 08:39:50 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.72       Driver Version: 410.72       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0   128W / 149W |   4569MiB / 11441MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
torch.cuda.is_available()

True

## Calculate mean and standard deviation 

In [5]:
transform_prep = tr.Compose([

    tr.Resize(64),
    tr.CenterCrop(64),
    tr.ToTensor()
])
loader_prep = torchvision.datasets.ImageFolder('./data/photomask_trainingdata_val/TRAIN/',
                                             transform=transform_prep)
data_prep = torch.utils.data.DataLoader(loader_prep, batch_size=111, shuffle=False, num_workers=4)

In [6]:
all_images_as_numpy = next(iter(data_prep))[0].numpy()

In [7]:
means = np.mean(all_images_as_numpy, axis=(0,2,3))
stds = np.std(all_images_as_numpy, axis=(0,2,3))
means,stds 

(array([0.48121634, 0.48121634, 0.48121634], dtype=float32),
 array([0.20189343, 0.20189343, 0.20189343], dtype=float32))

In [8]:
BATCH_SIZE = 8

## Prepare train data

In [9]:
augmentation_train = torchvision.transforms.Compose([
    tr.Resize((64)),
    tr.Pad((224-64)//2),
    tr.ColorJitter(hue=.05, saturation=.05),
    tr.RandomHorizontalFlip(),
    tr.RandomVerticalFlip(),
    tr.RandomRotation(20, resample=PIL.Image.BILINEAR),
    tr.ToTensor(),
    tr.Normalize(mean=means, std=stds)
])
loader = torchvision.datasets.ImageFolder('./data/photomask_trainingdata_val/TRAIN/',transform=augmentation_train)
data_train = torch.utils.data.DataLoader(loader,num_workers=4,batch_size=BATCH_SIZE,shuffle=True)
'Number of batches={}, number of images={}'.format(len(data_train),len(data_train)*BATCH_SIZE)

'Number of batches=12, number of images=96'

In [10]:
# next(iter(data_train))[0].shape

## Prepare validation data

In [11]:
augmentation_val = torchvision.transforms.Compose([
    tr.Resize((64)),
    tr.Pad((224-64)//2),
    tr.ColorJitter(hue=.05, saturation=.05),
    tr.RandomHorizontalFlip(),
    tr.RandomVerticalFlip(),
    tr.RandomRotation(20, resample=PIL.Image.BILINEAR),
    tr.ToTensor(),
    tr.Normalize(mean=means, std=stds)
])
loader = torchvision.datasets.ImageFolder('./data/photomask_trainingdata_val/VAL/',
                                             transform=augmentation_val)
data_val = torch.utils.data.DataLoader(loader,num_workers=4,batch_size=BATCH_SIZE)
'Number of batches={}, number of images={}'.format(len(data_val),len(data_val)*BATCH_SIZE)

'Number of batches=2, number of images=16'

In [13]:
test = next(iter(data_train))
test[0].shape,len(test[0])
image = torchvision.utils.make_grid(test[0])
image = np.transpose(image,(1,2,0))
# plt.imshow(image)

In [14]:
data = {'train':data_train,'val':data_val}
sizes = {'train':len(data_train)*BATCH_SIZE,'val':len(data_val)*BATCH_SIZE}

In [15]:
sizes

{'train': 96, 'val': 16}

In [16]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    history = {'val_loss':[],'train_loss':[],'val_acc':[],'train_acc':[]}

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in data[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            acc = running_corrects.double() / sizes[phase]
            loss =  running_loss / sizes[phase]
            epoch_loss = loss
            epoch_acc = acc
            
            if phase == 'val':
                history['val_loss'].append(loss)
                history['val_acc'].append(acc.item())
            else:
                history['train_loss'].append(loss)
                history['train_acc'].append(acc.item())
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since

    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model,history

In [17]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler


In [35]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show,save
output_notebook()

Loading BokehJS ...

In [19]:
import time
import copy


## Trying pretrained network

In [20]:
vgg = torchvision.models.vgg11(pretrained=True)

In [21]:
vgg.classifier[6].in_features

4096

In [48]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (device)

cuda:0


In [50]:
sf  = nn.Softmax()

In [23]:
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(vgg.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
vgg.classifier[6] = nn.Linear(4096, 3)

vgg = vgg.to(device)

In [24]:
model_ft,his = train_model(vgg, criterion, optimizer_ft,
                       exp_lr_scheduler,num_epochs=10)

Epoch 0/9
----------
train Loss: 0.9457 Acc: 0.6042
val Loss: 0.9269 Acc: 0.6250

Epoch 1/9
----------
train Loss: 0.7931 Acc: 0.6771
val Loss: 0.9316 Acc: 0.6250

Epoch 2/9
----------
train Loss: 0.6952 Acc: 0.7292
val Loss: 0.6923 Acc: 0.6250

Epoch 3/9
----------
train Loss: 0.5557 Acc: 0.7396
val Loss: 0.5704 Acc: 0.8125

Epoch 4/9
----------
train Loss: 0.4235 Acc: 0.8125
val Loss: 0.5070 Acc: 0.6875

Epoch 5/9
----------
train Loss: 0.4330 Acc: 0.7917
val Loss: 0.3409 Acc: 0.9375

Epoch 6/9
----------
train Loss: 0.3534 Acc: 0.8333
val Loss: 0.2958 Acc: 0.8125

Epoch 7/9
----------
train Loss: 0.3031 Acc: 0.8125
val Loss: 0.2990 Acc: 0.8125

Epoch 8/9
----------
train Loss: 0.2818 Acc: 0.8750
val Loss: 0.3140 Acc: 0.8750

Epoch 9/9
----------
train Loss: 0.2546 Acc: 0.8542
val Loss: 0.2564 Acc: 0.9375

Training complete in 1m 19s
Best val Acc: 0.937500


In [47]:
import torchsummary
torchsummary.summary(vgg,(3,224,244),BATCH_SIZE)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [8, 64, 224, 244]           1,792
              ReLU-2          [8, 64, 224, 244]               0
         MaxPool2d-3          [8, 64, 112, 122]               0
            Conv2d-4         [8, 128, 112, 122]          73,856
              ReLU-5         [8, 128, 112, 122]               0
         MaxPool2d-6           [8, 128, 56, 61]               0
            Conv2d-7           [8, 256, 56, 61]         295,168
              ReLU-8           [8, 256, 56, 61]               0
            Conv2d-9           [8, 256, 56, 61]         590,080
             ReLU-10           [8, 256, 56, 61]               0
        MaxPool2d-11           [8, 256, 28, 30]               0
           Conv2d-12           [8, 512, 28, 30]       1,180,160
             ReLU-13           [8, 512, 28, 30]               0
           Conv2d-14           [8, 512,

In [53]:
# create a new plot with default tools, using figure
p = figure(plot_width=600, plot_height=400, title="Loss")

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(range(len(his['val_loss'])),his['val_loss'], size=10, line_color="navy", fill_color="orange", fill_alpha=0.5)
p.line( range(len(his['val_loss'])),his['val_loss'], legend="val", line_width=2)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(range(len(his['train_loss'])),his['train_loss'], size=10, line_color="navy", fill_color="orange", fill_alpha=0.5)
p.line(range(len(his['train_loss'])),his['train_loss'], legend="train",line_width=2,line_color="red")
show(p) # show the results
# save(p,'./loss_vgg_0.94.html')

In [37]:
# create a new plot with default tools, using figure
p = figure(plot_width=600, plot_height=400, title="Accuracy")

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(range(len(his['val_acc'])),his['val_acc'], size=10, line_color="navy", fill_color="orange", fill_alpha=0.5)
p.line( range(len(his['val_acc'])),his['val_acc'], legend="val", line_width=2)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(range(len(his['train_acc'])),his['train_acc'], size=10, line_color="navy", fill_color="orange", fill_alpha=0.5)
p.line(range(len(his['train_acc'])),his['train_acc'], legend="train", line_width=2,line_color="red")
show(p) # show the results
save(p,'./acc_vgg_0.94.html')

'/home/vladimir_danilov0000/acc_vgg_0.94.html'

In [38]:
torch.save(model_ft.state_dict(), './acc_vgg_0.94.pth')


In [39]:
vgg.load_state_dict(torch.load('./acc_vgg_0.94.pth'))

In [40]:
vgg = vgg.to(device)

In [45]:
running_loss = 0.0
running_corrects = 0
for inputs, labels in data_val:
    inputs = inputs.to(device)
    labels = labels.to(device)
    # zero the parameter gradients
#     optimizer.zero_grad()
    with torch.set_grad_enabled(False):
        outputs = vgg(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
         # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)


In [46]:
running_loss/sizes['val'],running_corrects.item()/sizes['val']

(0.3714132755994797, 0.9375)